In [2]:
import  numpy  as  np
import  tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

mnist = input_data.read_data_sets('../mnist/', one_hot=True)

Extracting ../mnist/train-images-idx3-ubyte.gz
Extracting ../mnist/train-labels-idx1-ubyte.gz
Extracting ../mnist/t10k-images-idx3-ubyte.gz
Extracting ../mnist/t10k-labels-idx1-ubyte.gz


In [7]:
#Init weights
def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [8]:
#Init biases
def init_biases(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

In [9]:
#Conv2D
def conv2d(x, weights):
    # x --> [batch_size, height, width, channels]
    # weights -> [filter_height, filter_width, channels_in, channels_out]
    
    return tf.nn.conv2d(x, weights, strides=[1,1,1,1], padding='SAME') #'SAME' - 0 padding

In [11]:
#MaxPooling
def max_pooling(x, k):
    # x --> [batch_size, height, width, channels]
    # k --> height(width) size
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

In [12]:
#Convolutional layer
def conv_layer(input_x, shape):
    W = init_weights(shape)
    b = init_biases([shape[3]])
    
    return tf.nn.relu(conv2d(input_x, W)+b)

In [13]:
#Fully Connected
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_weights([size])
    
    return tf.matmul(input_layer, W) + b

In [14]:
#Placeholder
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

In [15]:
#Layers
x_image = tf.reshape(x, shape=[-1,28,28,1])

conv1 = conv_layer(x_image, shape=[5,5,1,32]) #5x5 - patch size, 1 - input channel(grayscale), 32 - output channels
pool1 = max_pooling(conv1, 2)

conv2 = conv_layer(pool1, shape=[5,5,32,64])
pool2 = max_pooling(conv2, 2)

conv_to_flat = tf.reshape(pool2, shape=[-1, 7*7*64])
fc1 = tf.nn.relu(normal_full_layer(conv_to_flat, 1024))

In [16]:
#Dropout
keep_prob = tf.placeholder(dtype=tf.float32)
dropout = tf.nn.dropout(fc1, keep_prob=keep_prob)

In [18]:
y_pred = normal_full_layer(dropout, 10)

In [19]:
#Loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y))

#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

#Init
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(5000):
        batch_x, batch_y = mnist.train.next_batch(100)
        
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob:0.5})
        
        if epoch % 100 == 0:
            print('[%d/%d]' % (epoch+1, 5000))
            correct_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))
            print('Accuracy on test set: ', sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob:1.0}))

[1/5000]
Accuracy on test set:  0.1388
[101/5000]
Accuracy on test set:  0.9516
[201/5000]
Accuracy on test set:  0.967
[301/5000]
Accuracy on test set:  0.9678
[401/5000]
Accuracy on test set:  0.9804
[501/5000]
Accuracy on test set:  0.9787
[601/5000]
Accuracy on test set:  0.9821
[701/5000]
Accuracy on test set:  0.9839
[801/5000]
Accuracy on test set:  0.9833
[901/5000]
Accuracy on test set:  0.9836
[1001/5000]
Accuracy on test set:  0.9875
[1101/5000]
Accuracy on test set:  0.9859
[1201/5000]
Accuracy on test set:  0.9863
[1301/5000]
Accuracy on test set:  0.9864
[1401/5000]
Accuracy on test set:  0.9901
[1501/5000]
Accuracy on test set:  0.9895
[1601/5000]
Accuracy on test set:  0.9893
[1701/5000]
Accuracy on test set:  0.9892
[1801/5000]
Accuracy on test set:  0.99
[1901/5000]
Accuracy on test set:  0.9891
[2001/5000]
Accuracy on test set:  0.9884
[2101/5000]
Accuracy on test set:  0.9896
[2201/5000]
Accuracy on test set:  0.9858
[2301/5000]
Accuracy on test set:  0.9901
[2401/5